In [1]:
import h5py
import numpy as np
import os
import random
from os.path import join
from keras import layers
from keras.models import Sequential
from keras.layers import Conv2D, Dense, MaxPooling2D, Dropout, Flatten, Activation
from keras.utils import to_categorical

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
dataset = 'UCF11'
nb_classes = int(dataset[3:])

In [3]:
def preprocess_images(images):

    # InceptionV3 requires images to be in range -1 to 1.
    return ((images / 255.) - 0.5) * 2

In [4]:
def train_generator(dataset_dir, batch_size):
    
    all_files = os.listdir(dataset_dir)
    
    while True:
        images = []
        labels = []

        for _ in range(batch_size):
            
            random_filename = random.choice(all_files)
            
            with h5py.File(join(dataset_dir, random_filename), 'r') as hf:
                frames = hf['X'][:]
                fr_labels = hf['Y'][:]
                random_idx = np.random.randint(frames.shape[0])
                
                images.append(frames[random_idx])
                labels.append(fr_labels)
                
        images = np.array(images)
        labels = to_categorical(np.array(labels), nb_classes)
        
        yield preprocess_images(images), labels
        
        

def valid_generator(dataset_dir, batch_size):
    
    all_files = os.listdir(dataset_dir)
    
    while True:
        
        for filename in all_files:

            with h5py.File(join(dataset_dir, filename), 'r') as hf:
                frames = hf['X'][:]
                single_label = hf['Y'][:][0]

                fr_labels = np.array([single_label] * frames.shape[0])

                yield preprocess_images(frames), to_categorical(fr_labels, nb_classes)

In [5]:
separate_dataset_dir = join('datasets', dataset, 'separate_frames_50_h_240_w_320')
train_dir = join(separate_dataset_dir, 'train')
valid_dir = join(separate_dataset_dir, 'valid')

train_samples_count = len(os.listdir(train_dir))
valid_samples_count = len(os.listdir(valid_dir))

with h5py.File(join(train_dir, os.listdir(train_dir)[0])) as hf:
    image_shape = hf['X'][:].shape[1:]
    print('Image input shape is', image_shape)

Image input shape is (240, 320, 3)


In [6]:
def build_model():
    model = Sequential()
    model.add(Conv2D(128, (3, 3), padding='same',
                     input_shape=image_shape))
    model.add(Activation('relu'))
    model.add(Conv2D(128, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(256, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(256, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(nb_classes))
    model.add(Activation('softmax'))
    
    return model

In [7]:
batch_size = 32

from keras import metrics
from functools import partial

top_3_k_categorical_accuracy = partial(metrics.top_k_categorical_accuracy, k=3)
top_3_k_categorical_accuracy.__name__ = 'top_3'

model = build_model()
print(model.summary())
model.compile(optimizer='adam', loss='categorical_crossentropy', 
              metrics=['accuracy', top_3_k_categorical_accuracy])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 240, 320, 128)     3584      
_________________________________________________________________
activation_1 (Activation)    (None, 240, 320, 128)     0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 238, 318, 128)     147584    
_________________________________________________________________
activation_2 (Activation)    (None, 238, 318, 128)     0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 119, 159, 128)     0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 119, 159, 128)     0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 119, 159, 256)     295168    
__________

In [ ]:
history = model.fit_generator(train_generator(train_dir, batch_size),
                    steps_per_epoch=train_samples_count * 50 // batch_size, 
                    validation_data=valid_generator(valid_dir, batch_size),
                    validation_steps=valid_samples_count,
                    epochs=3)

Epoch 1/3
1478/1478 [==============================] - 1034s 699ms/step - loss: 0.5713 - acc: 0.8115 - top_3: 0.9237 - val_loss: 3.5669 - val_acc: 0.3997 - val_top_3: 0.6463
Epoch 2/3
1478/1478 [==============================] - 1027s 695ms/step - loss: 0.1215 - acc: 0.9623 - top_3: 0.9955 - val_loss: 4.1268 - val_acc: 0.4286 - val_top_3: 0.6564
Epoch 3/3
  19/1478 [..............................] - ETA: 14:55 - loss: 0.0977 - acc: 0.9753 - top_3: 0.9951

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure(figsize=(12, 8))
plt.grid()
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy (training with SGD)')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
batch_size = 16

from keras import metrics
from functools import partial

top_3_k_categorical_accuracy = partial(metrics.top_k_categorical_accuracy, k=3)
top_3_k_categorical_accuracy.__name__ = 'top_3'

model = build_model()
print(model.summary())
model.compile(optimizer='adam', loss='categorical_crossentropy', 
              metrics=['accuracy', top_3_k_categorical_accuracy])

In [ ]:
history = model.fit_generator(train_generator(train_dir, batch_size),
                    steps_per_epoch=train_samples_count * 50 // batch_size, 
                    validation_data=valid_generator(valid_dir, batch_size),
                    validation_steps=valid_samples_count,
                    epochs=20)